`Write an pyspark code to find the ctr of each Ad.Round ctr to 2
decimal points. Order the result table by ctr in descending order
and by ad_id in ascending order in case of a tie.`

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *  # Import the function
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.functions import col, explode, array_repeat
import json
spark = SparkSession.builder.getOrCreate()

In [4]:
schema = StructType([
    StructField('ad_id', IntegerType(), True),
    StructField('user_id', IntegerType(), True),
    StructField('action', StringType(), True)
    
])
data = [
(1, 1, 'Clicked'),
(2, 2, 'Clicked'),
(3, 3, 'Viewed'),
(5, 5, 'Ignored'),
(1, 7, 'Ignored'),
(2, 7, 'Viewed'),
(3, 5, 'Clicked'),
(1, 4, 'Viewed'),
(2, 11, 'Viewed'),
(1, 2, 'Clicked')
]

In [5]:
df = spark.createDataFrame(data, schema)
df.show()

+-----+-------+-------+
|ad_id|user_id| action|
+-----+-------+-------+
|    1|      1|Clicked|
|    2|      2|Clicked|
|    3|      3| Viewed|
|    5|      5|Ignored|
|    1|      7|Ignored|
|    2|      7| Viewed|
|    3|      5|Clicked|
|    1|      4| Viewed|
|    2|     11| Viewed|
|    1|      2|Clicked|
+-----+-------+-------+



In [6]:
ctr_df = df.groupBy(col('ad_id')).agg(
    sum(when(col('action') == 'clicked', 1).otherwise(0)).alias('click_count'),
    sum(when(col('action') == 'Viewed', 1).otherwise(0)).alias('view_count'),
)
#ctr_df = ctr_df.withColumn('ctr', round(col('click_count') / (col('click_count') + col('view_count')), 2))
#ctr_df = ctr_df.withColumn("ctr", round(col("click_count") / (col("click_count") + col("view_count")), 2))

ctr_df = ctr_df.withColumn(
    "ctr",
    when(
        (col("click_count") + col("view_count")) == 0, 0
    ).otherwise(
        round(
            col("click_count") / (col("click_count") + col("view_count")),
            2
        )
    ) 
)





ctr_df.show()

+-----+-----------+----------+---+
|ad_id|click_count|view_count|ctr|
+-----+-----------+----------+---+
|    1|          0|         1|0.0|
|    3|          0|         1|0.0|
|    5|          0|         0|0.0|
|    2|          0|         2|0.0|
+-----+-----------+----------+---+

